# **Running the best GPS with KNN Generate**
<a href="https://colab.research.google.com/github/raphaelrubrice/BornToOverfit/blob/raph/Precomputed_CHEMBERT_GPS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Colab setup**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
# to avoid having the data on your drive
%cd /content

Mounted at /content/drive
/content


In [ ]:
!git clone https://github.com/raphaelrubrice/BornToOverfit.git
%cd BornToOverfit
!git checkout raph

Cloning into 'BornToOverfit'...
remote: Enumerating objects: 550, done.
remote: Counting objects: 100% (131/131), done.
remote: Compressing objects: 100% (89/89), done.
remote: Total 550 (delta 80), reused 78 (delta 42), pack-reused 419 (from 1)
Receiving objects: 100% (550/550), 392.19 KiB | 17.05 MiB/s, done.
Resolving deltas: 100% (359/359), done.
/content/BornToOverfit
Branch 'Camille' set up to track remote branch 'Camille' from 'origin'.
Switched to a new branch 'Camille'


Installing requirements

In [ ]:
!pip install -r data_baseline/requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 73.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.4/36.4 MB 76.9 MB/s eta 0:00:00


**Downloading the data**

In [ ]:
!cp -r /content/drive/MyDrive/MVA/ALTEGRAD/challenge_data/data /content/BornToOverfit/data_baseline/.

cp: cannot open '/content/drive/MyDrive/Kaggle_ALTEGRAD/data/results/baseline.gsheet' for reading: Operation not supported


## **KNN Generate**

Add Molecule cards

In [ ]:
!python data_baseline/data_utils.py

Load embeddings and models

In [ ]:
!cp -r /content/drive/MyDrive/MVA/ALTEGRAD/challenge_data/Cam_Raph /content/BornToOverfit/data_baseline/.

In [ ]:
import os
path_to_folder = "/content/BornToOverfit/data_baseline/Cam_Raph/"
path_to_prototype = "/content/BornToOverfit/prototype/"

os.environ['SAVING_FOLDER'] = path_to_prototype + "test-2_dual_lrGNN0.0008_lrBERT3e-05_wd0.0001_frz0_margin0.2_bs128/"
os.environ['SAVING_SUBFOLDER'] = "knn_llm"
os.environ['SAVING_SFT_SUBFOLDER'] = os.environ['SAVING_SUBFOLDER'] + "_sft"
os.environ['GNN_PATH'] = path_to_folder + "test-2_dual_lrGNN0.0008_lrBERT3e-05_wd0.0001_frz0_margin0.2_bs128.pt"
os.environ['TRAIN_EMB_PATH'] = path_to_folder + "train_embeddings_RealChemBERT.pt"
os.environ['VAL_EMB_PATH'] = path_to_folder + "validation_embeddings_RealChemBERT.pt"

Create merged TRAIN/VAL data

In [ ]:
os.environ['DATA_FOLDER'] = "/content/BornToOverfit/data_baseline/data"
os.environ['MERGED_GRAPHS'] = os.environ['DATA_FOLDER'] + "/train_val_graphs.pkl"
!python data_baseline/merge_embeddings.py \
  --train_path $TRAIN_EMB_PATH \
  --val_path $VAL_EMB_PATH \
  --output_dir $DATA_FOLDER

In [ ]:
os.environ['MERGED_EMB'] = os.environ['DATA_FOLDER'] + "/train_val_embeddings.pt"
!python data_baseline/merge_embeddings.py \
  --train_path $TRAIN_EMB_PATH \
  --val_path $VAL_EMB_PATH \
  --output_dir $DATA_FOLDER

Use all data ?

In [ ]:
USE_ALL_DATA = False
if USE_ALL_DATA:
    os.environ['TRAINING_DATA'] = os.environ['MERGED_EMB']
else:
    os.environ['TRAINING_DATA'] = os.environ['TRAIN_EMB_PATH']

Fine-tune the LLM with SFT

In [ ]:
!python prototype/knn_tanimoto_generate.py -f_data data_baseline/data -f $SAVING_FOLDER \
  -gnn_path $GNN_PATH \
  -train_emb_path $TRAINING_DATA \
  --do_finetune \
  --do_sft \
  --k 5 \
  --use_lora \
  --sft_steps 500 \
  --sft_lora 128 \
  --sft_neftune \
  --per_device_bs 1 \
  --grad_accum 8 \
  --base_llm deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B \
  --out_llm_dir $SAVING_SUBFOLDER \
  --max_prompt_length 3096

Generate on validation with the fine-tuned model + evaluation metrics

In [ ]:
!python prototype/knn_tanimoto_generate.py -f_data data_baseline/data -f $SAVING_FOLDER \
  -gnn_path $GNN_PATH \
  -train_emb_path $TRAINING_DATA \
  --do_generate --split validation --k 5 \
  --gen_batch_size 8 \
  --out_llm_dir $SAVING_SFT_SUBFOLDER \
  --max_prompt_length 3096

Generate on test (no references; no evaluation)

In [ ]:
!python prototype/knn_tanimoto_generate.py -f_data data_baseline/data -f $SAVING_FOLDER \
  -gnn_path $GNN_PATH \
  -train_emb_path $TRAINING_DATA \
  --do_generate --split test --k 5 \
  --gen_batch_size 8 \
  --out_llm_dir $SAVING_SFT_SUBFOLDER \
  --max_prompt_length 3096

See generated data on the test set

In [ ]:
import pandas as pd
os.environ['FINAL_SAVING_FOLDER'] = os.environ['SAVING_FOLDER'][:-1]
test = pd.read_csv(os.environ['FINAL_SAVING_FOLDER'] + "/test_knn_gen_k5.csv")
test

In [ ]:
print(test.iloc[18]["Prompt"])
print(test.iloc[18]["generated_description"])

Saving to Drive

In [ ]:
!cp -r $FINAL_SAVING_FOLDER /content/drive/MyDrive/MVA/ALTEGRAD/challenge_data/.